<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/DDMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd
from google.colab import files
import requests

# Read the news for AAPL stock to assess risks
#news_url = "https://newsapi.org/v2/everything?q=AAPL&apiKey=YOUR_API_KEY"
news_url = "https://newsapi.org/v2/everything?q=AAPL&apiKey=7f4f6db648674bb8afbcdb7567c7b91d"

response = requests.get(news_url)
news_data = response.json()

print("Latest News for AAPL Stock:\n")
for article in news_data['articles'][:5]:  # Displaying top 5 news articles
    print(f"Title: {article['title']}")
    print(f"Description: {article['description']}\n")

# Load the CSV data
# 0. Read the CSV file:

uploaded = files.upload()
filename = list(uploaded.keys())[0]  # Assuming you uploaded only one file
df = pd.read_csv(filename)

df = pd.read_csv('AAPLData.csv')

# Convert the 'Implied Volatility' column from percentage to a decimal
df['Implied Volatility'] = df['Implied Volatility'].str.rstrip('%').astype('float') / 100.0

# Extract matrix M from the dataframe
M = df[['Strike', 'Implied Volatility']].values

# 1. Matrix Representation:

# Current stock price for each option
X = np.array([[150] for _ in range(len(df))])

# Expected future stock price (for simplicity, let's assume a 10% increase)
Y = 1.1 * X

# 2. Price Prediction:

# For simplicity, let's assume a linear model without bias: Y = AX
A = Y / X

# 3. Option Payoff:

# Using a simple call option payoff: max(Y - strike_price, 0)
payoffs = np.maximum(Y - M[:, 0].reshape(-1, 1), 0)

# 4. Optimization:

# Let's optimize for maximum expected return
# We'll use linear programming to find the best weights for each option in the portfolio
c = -payoffs.ravel()  # We negate because linprog tries to minimize, not maximize
bounds = [(0, 1) for _ in range(payoffs.shape[0])]  # weights between 0 and 1
result = linprog(c, bounds=bounds)
weights = result.x

# 5. Risk Management:

# Compute the delta for each option
small_increase = 0.01
deltas = (np.maximum((Y + small_increase) - M[:, 0].reshape(-1, 1), 0) - payoffs) / small_increase

# Delta Hedging: If you have a portfolio of options, the net delta will tell you how exposed your portfolio is to changes in the stock price.
net_delta = np.sum(deltas * weights)

# 6. Arbitrage Opportunities:

# Identify options with payoffs significantly higher than their current price
arbitrage_opportunities = np.where(payoffs.ravel() > M[:, 0])[0]

# 7. Scenario Analysis:

# Consider two scenarios: a 5% increase and a 5% decrease in stock price
Y_increase = 1.05 * X
Y_decrease = 0.95 * X

payoffs_increase = np.maximum(Y_increase - M[:, 0].reshape(-1, 1), 0)
payoffs_decrease = np.maximum(Y_decrease - M[:, 0].reshape(-1, 1), 0)

print("Original Payoffs:", payoffs.ravel())
print("Payoffs with 5% increase:", payoffs_increase.ravel())
print("Payoffs with 5% decrease:", payoffs_decrease.ravel())
print("Net Delta of Portfolio:", net_delta)

# Trading Advice based on the results

print("\nAAPL Options Trading Advice:\n")

print("1. Iron Condor Strategy:")
print("- The Iron Condor is a neutral strategy, ideal for minimal stock movement.")
print("- Sell an out-of-the-money (OTM) call and an OTM put.")
print("- Buy a further OTM call and a further OTM put to cap potential losses.")
print("- Aim for the stock price to remain between the strike prices of the sold options at expiration.\n")

print("2. Covered Call Strategy:")
print("- If owning AAPL shares, consider selling calls against them for additional income.")
print("- Sell calls with strike prices at the higher end of the range for potential profit.\n")

print("3. Protective Put Strategy:")
print("- If concerned about a potential decrease in AAPL's price, consider buying puts as insurance.")
print("- Buy puts with strike prices where potential payoffs start to increase for protection.\n")

print("4. Vertical Spread:")
print("- For a bullish outlook, use a bull call spread: Buy a call at a lower strike and sell a call at a higher strike.")
print("- For a bearish outlook, use a bear put spread: Buy a put at a higher strike and sell a put at a lower strike.\n")

print("Final Thoughts:")
print("Consider current market conditions, upcoming events impacting AAPL, and personal risk tolerance before trading. Consult with a financial advisor for tailored advice.")

print("AAPL Stock Options Trading Advice:\n")

##############################

print("AAPL Stock Options Trading Advice:\n")

# Approximate current stock price
current_stock_price = df.loc[df['Strike'].idxmin()]['Last Price']

# Identify high liquidity options
high_liquidity_calls = df[(df['Volume'] > df['Volume'].mean()) & (df['Strike'] > current_stock_price)].sort_values(by='Volume', ascending=False).head(1)
high_liquidity_puts = df[(df['Volume'] > df['Volume'].mean()) & (df['Strike'] < current_stock_price)].sort_values(by='Volume', ascending=False).head(1)

print(f"The current approximate stock price of AAPL is ${current_stock_price:.2f}.")

# Assessing the Net Delta of the Portfolio
if net_delta > 0:
    print("The net delta of the portfolio is positive, indicating a bullish sentiment on AAPL.")
    print("\nSuggested Strategy: Consider a bullish strategy.")
    if not high_liquidity_calls.empty:
        print(f"Consider buying the call option with a strike price of ${high_liquidity_calls['Strike'].values[0]:.2f} due to its high liquidity. The current bid is ${high_liquidity_calls['Bid'].values[0]:.2f} and the ask is ${high_liquidity_calls['Ask'].values[0]:.2f}.")
    if not high_liquidity_puts.empty:
        print(f"Additionally, consider selling the put option with a strike price of ${high_liquidity_puts['Strike'].values[0]:.2f} to generate premium income. The current bid is ${high_liquidity_puts['Bid'].values[0]:.2f} and the ask is ${high_liquidity_puts['Ask'].values[0]:.2f}.")
else:
    print("The net delta of the portfolio is negative, indicating a bearish sentiment on AAPL.")
    print("\nSuggested Strategy: Consider a bearish strategy.")
    if not high_liquidity_puts.empty:
        print(f"Consider buying the put option with a strike price of ${high_liquidity_puts['Strike'].values[0]:.2f} due to its high liquidity. The current bid is ${high_liquidity_puts['Bid'].values[0]:.2f} and the ask is ${high_liquidity_puts['Ask'].values[0]:.2f}.")
    if not high_liquidity_calls.empty:
        print(f"Additionally, consider selling the call option with a strike price of ${high_liquidity_calls['Strike'].values[0]:.2f} to generate premium income. The current bid is ${high_liquidity_calls['Bid'].values[0]:.2f} and the ask is ${high_liquidity_calls['Ask'].values[0]:.2f}.")

# Iron Condor Strategy based on Implied Volatility
if 'Implied Volatility' in df.columns:
    df['Implied Volatility'] = df['Implied Volatility'].astype(str)
    average_iv = df['Implied Volatility'].str.rstrip('%').astype('float').mean()
    if average_iv > 50:
        print("\nGiven the high implied volatility, consider implementing an Iron Condor strategy. This involves selling an out-of-the-money call and put option, while simultaneously buying a further out-of-the-money call and put option. This strategy allows you to generate premium income while limiting potential losses.")
    else:
        print("\nGiven the relatively low implied volatility, it might not be the best time to implement an Iron Condor strategy. Instead, focus on directional strategies based on your market outlook.")
else:
    print("\nUnable to determine implied volatility. Please check the data.")


"Delta-Driven Matrix Optimization (DDMO) Strategy"

Description:
The Delta-Driven Matrix Optimization (DDMO) Strategy is a comprehensive approach to options trading that combines the predictive power of delta with a matrix-based analysis of potential payoffs. This strategy is designed to provide traders with a holistic view of potential outcomes while also offering actionable insights based on current market conditions.

Key Components:

Delta Analysis: The net delta of the portfolio provides an indication of the overall market sentiment towards the underlying stock. A positive net delta indicates a bullish sentiment, while a negative net delta suggests a bearish sentiment. This analysis helps in determining the overall direction of the trading strategy.

Matrix-Based Payoff Analysis: This component involves creating a matrix of potential payoffs under various scenarios, such as a 5% increase or decrease in the stock price. By visualizing the potential outcomes in a matrix format, traders can better understand the risk-reward profile of their portfolio and make informed decisions.

Optimization: Based on the matrix analysis and delta, the strategy provides specific actionable advice, such as selling or buying particular options, to optimize potential earnings and reduce potential losses.

Liquidity Consideration: The strategy also takes into account the liquidity of options (based on bid-ask spread and volume) to ensure that the recommended trades can be executed efficiently in the market.

Benefits:

Provides a comprehensive view of potential outcomes.
Offers actionable insights based on current market sentiment.
Helps in risk management by visualizing potential payoffs.
Considers liquidity, ensuring feasible trade execution.
The DDMO Strategy is suitable for traders who seek a balanced approach, combining both predictive analysis (via delta) and a comprehensive understanding of potential outcomes (via the matrix). It's especially beneficial for those looking to optimize their options portfolio in dynamic market conditions.